In [1]:
import video_processing as vp
import matplotlib.pyplot as plt
import seed
import numpy

## Static global variables and file names

In [ ]:
HEIGHT = 256
WIDTH = 256
TRUE_FRAMERATE = 28.815

mouse_brain_raw = ""

## Create Bregma from coordinates

In [4]:
image = numpy.zeros((256,256), dtype=numpy.uint8)
bregma = seed.Bregma.from_coordinates(column=147, row=133)

Bregma Location:
Column 147
Row 133


In [ ]:
seeds = seed.generate_scaled_seeds(
    seeds=seeds,
    bregma=bregma,
    ppm=27.6,
    direction="u"
)

## Plot locations of seeds and bregma

In [ ]:
normal_seeds_x_R = []
normal_seeds_y_R = []
normal_seeds_x_L = []
normal_seeds_y_L = []
image[br.row, br.col] = 255
for seed in seeds:
    if "-R" in seed.name:
        image[seed.row, seed.col] = 255
        normal_seeds_x_R.append(seed.col)
        normal_seeds_y_R.append(seed.row)
    else:
        image[seed.row, seed.col] = 128
        normal_seeds_x_L.append(seed.col)
        normal_seeds_y_L.append(seed.row)
    
seeds_fig = plt.figure(figsize=(8, 8))
seeds_ax = seeds_fig.add_subplot(111)
seeds_ax.matshow(image)
seeds_ax.plot(bregma.col, bregma.row, '.k')
seeds_ax.plot(normal_seeds_x_L, normal_seeds_y_L, '.b')
seeds_ax.plot(normal_seeds_x_R, normal_seeds_y_R, '.r')

## Extract RAW frames, calculate $\frac{\Delta F}{F_0}$ and apply chebychev type I bandpass filter

In [ ]:
mouse_green_frames = vp.extract_RAW_frames(
    mouse_brain_raw,
    width=WIDTH,
    height=HEIGHT,
    channel='green'
)

In [ ]:
#var_green_frames = numpy.var(mouse_green_frames, axis=0)
#mean_green_frames = numpy.mean(mouse_green_frames, axis=0)
mouse_green_frames, var_green_frames = vp.calculate_df_f0(mouse_green_frames)

In [6]:
low_limit = 0.01
high_limit = 12.0
cheby1_params = vp.Filter(low_limit, high_limit, TRUE_FRAMERATE, order=3)
mouse_green_frames = cheby1_params.filter(mouse_green_frames)

## Generate correlation matrix

In [3]:
bregma = seed.Bregma.from_coordinates(140, 133)
ppmm = 26.5
seeds = (
    ("M1", 1.03 + 1, 1.365),
    ("FL", 0.16, 2.47),
    ("HL", -0.7, 1.85),
    ("aBC", -1.36 + 0.575, 3.35),
    ("pBC", -1.9, 3.35),
    ("AC", 0 + 1, 0.6),
    ("RS", -2.8858 + 1, 0.62043),
    ("V1", -4.2678 + 0.8, 2.5168),
    ("mPTA", -2.4962, 2.2932),
    ("lPTA", -2.4962 - 0.3, 3.35 - 0.2),
    ("Un", -1.5, 2.6),
)
scaled_seeds = seed.generate_scaled_seeds(
    seeds, bregma, ppm
)


Bregma Location:
Column 140
Row 133


In [ ]:
seed.generate_correlation_matrix(
    l_mouse_frames=mouse_green_frames, 
    r_mouse_frames=mouse_green_frames, # pass truncated frames
    l_seeds=scaled_seeds,
    r_seeds=scaled_seeds,
    title="Correlation matrix for one mouse",
    filename="correlation_matrix",
    radius=5,
    interpolation="nearest",
    cmap="viridis",
    figsize=(10, 11)
)

In [4]:
def plot_seed_pixel_gallery(
    seeds,
    seed_frames,
    other_frames,
    title,
    ncols=5,
    figsize=(13, 13),
    cmap="viridis",
):
    # for each of the speed pixels, generate a correlation map
    num_seeds = len(seeds)
    _, height, width = other_frames.shape
    correlation_maps = numpy.zeros(
        (num_seeds, height, width)
    )
    for seed_index, seed in enumerate(seeds):
        seed_signal = seed_frames[:, seed.row, seed.col]
        for i in range(height):
            for j in range(width):
                correlation_maps[
                    seed_index, i, j
                ] = pearsonr(
                    other_frames[:, i, j], seed_signal
                )
    fig = plt.figure(figsize=figsize)
    nrows = num_seeds / ncols
    if int(nrows) == nrows:
        nrows = int(nrows)
    else:
        nrows = int(nrows) + 1
    # generate the plots
    for i, correlation_map in enumerate(correlation_maps):
        ax = fig.subplot(nrows, ncols, i + 1)
        plot = ax.matshow(
            correlation_map, vmin=0.0, vmax=1.0, cmap=cmap
        )
        ax.set_title(seeds[i].name)
        fig.colorbar(plot, ax=ax, fraction=0.046, pad=0.04)
        ax.get_yaxis().set_visible(False)
        ax.get_xaxis().set_visible(False)
    fig.suptitle(title)

In [ ]:
plot_seed_pixel_gallery(
    seeds, 
    mouse_green_frames,
    mouse_green_frames,
    "Seed Pixel Gallery",
)